<a href="https://colab.research.google.com/github/human-ai2025/Elo-Merchant-Recommendation/blob/master/2_merging_data_elo_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [1]:
#Import Libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import gc
from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
import datetime
import time
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import mode

### Mount the drive 

In [2]:
#Mounting drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Setup the current path 

In [3]:
!ls

drive  sample_data


In [4]:
#Setting up worksapce directory 
%cd /content/drive/MyDrive/data 

/content/drive/MyDrive/data


In [5]:
!ls

 augmented_test.csv			  new_merchant_transactions.csv
 augmented_train.csv			  sample_submission.csv
'Data Dictionary.xlsx'			  test.csv
 Data_Dictionary.xlsx			  train.csv
 historical_transactions.csv		  transactions_raw_merged.csv
 merchants.csv				  transactions_refined_1_merged_WON.csv
 merged_transactions_with_merchants.csv


## Timer 

In [6]:
#refer:-https://www.youtube.com/watch?v=vOMtQ4ocMGI
@contextmanager
def timer(title):
    """ used to calculate time for each function"""
    t0 = time.time()
    yield
    print("{} - done in {:.000f}s".format(title, time.time() - t0))

## Memory Reduction 

In [7]:
#https://www.kaggle.com/fabiendaniel/elo-world
#Function to load data into pandas and reduce memory usage

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    #loop for alll the columns in the dataframe 
    for col in df.columns:
        #get the datatype of the column
        col_type = df[col].dtypes
        #if the data type is numeric then only start changing the datatype
        #as it isnt much helpful for other data types 
        if col_type in numerics:
            #stores the min value of the column 
            c_min = df[col].min()
            #stores the maximum value of the column
            c_max = df[col].max()
            #for int type numerics
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            #for float type numerics 
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## One hot encoder 

In [8]:
#Refer:-https://stackoverflow.com/questions/36631163/what-are-the-pros-and-cons-between-get-dummies-pandas-and-onehotencoder-sciki
def one_hot_encoder(df, nan_as_category=True):
    """used to create the one hot encoding of the categorical variables """
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    print("The columns on which one hot encoding is performed is ",categorical_columns)
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

## FE on Train Test

In [9]:
def train_test(num_rows=None):

    # load csv
    train_df = pd.read_csv('/content/drive/MyDrive/data/train.csv', index_col=['card_id'], nrows=num_rows)
    test_df = pd.read_csv('/content/drive/MyDrive/data/test.csv', index_col=['card_id'], nrows=num_rows)

    print("Train samples: {}, test samples: {}".format(len(train_df), len(test_df)))
    OUTLIER_THRESHOLD = 30
    # Create an outliers column set to 1 for
    train_df['outliers'] = np.where(train_df['target'] < OUTLIER_THRESHOLD, 1, 0)

    # set target as nan
    test_df['target'] = np.nan

    # merge
    df = train_df.append(test_df)

    del train_df, test_df
    gc.collect()

    # to datetime
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])

    # datetime features
    df['quarter'] = df['first_active_month'].dt.quarter
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['quarter_first_active_month'] = df['first_active_month'].dt.quarter
    df['first_active_month_diff_from_today'] = (datetime.datetime.today() - df['first_active_month']).dt.days


    # one hot encoding
    df, cols = one_hot_encoder(df, nan_as_category=False)

    for col in ['feature_1', 'feature_2', 'feature_3']:
        order_label = df.groupby(col)['outliers'].mean()
        df[col] = df[col].map(order_label)

    # Some basic statistics transformations over the feature_i columns
    df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
    df['feature_mean'] = df['feature_sum'] / 3
    df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
    df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
    df['feature_std'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

    return df

## Imputations on transactions data 

In [10]:
def transactions_imputations(df):
    """
    Data preprocessing consists of 
    1. Handling missing values 
    2. Mapping of categorical columns
    """

    # fillna
    df['category_2'].fillna(6, inplace=True)
    df['category_3'].fillna(3, inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a', inplace=True)
    df['installments'].replace(-1, np.nan, inplace=True)
    df['installments'].replace(999, np.nan, inplace=True)
    df['installments'].fillna(df['installments'].mode()[0], inplace=True)



    # mapping categorical to numerical 
    df['authorized_flag'] = df['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    df['category_1'] = df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    df['category_3'] = df['category_3'].map({'A': 0, 'B': 1,'C': 2, 3:3}).astype(int)
    df['category_2'] = df['category_2'].map({1.0 : 1, 2.0 : 2, 3.0 : 3,4.0 : 4, 5.0 : 5, 6 : 6}).astype(int)
    #removing purchase amount outliner    
    df['purchase_amount'] = df['purchase_amount'].apply(lambda x: min(x, 0.8))
    df['price'] = df['purchase_amount'] / (df['installments'] + 0.001) #some epsilone for 0  installments 

    # reduce memory usage
    #df = reduce_mem_usage(df)

    return df


## Imputations on Merchant Data

In [27]:
def merchant_imputations(num_rows=None):
    """
    Data preprocessing consists of 
    1. Handling missing values 
    2. Mapping of categorical columns
    """
    # load csv
    df = pd.read_csv('/content/drive/MyDrive/data/merchants.csv', nrows=num_rows)

    #drop duplicate merchant id
    df.drop_duplicates(subset=['merchant_id'], keep='last')

    df['category_1'] = df['category_1'].fillna(2)
    df['category_1'] = df['category_1'].map({'Y': 0, 'N': 1,2:2}).astype(int)

    #for most_recent_sales_range_merchants_t_merchants_t and most_recent_purchases_range_merchants_t_merchants_t
    #it has A,B,C,D,E and null values 
    df['most_recent_sales_range'] = df['most_recent_sales_range'].fillna(5)
    df['most_recent_purchases_range'] = df['most_recent_purchases_range'].fillna(5)
    df['most_recent_sales_range'] = df['most_recent_sales_range'].map({'A': 0, 'B': 1,'C': 2, 'D': 3,'E' : 4, 5 : 5}).astype(int)
    df['most_recent_purchases_range'] = df['most_recent_purchases_range'].map({'A': 0, 'B': 1,'C': 2, 'D': 3,'E' : 4, 5 : 5}).astype(int)
    

    #for category_4_merchants_t_merchants_t
    #it has Y,N and null values 
    df['category_4'] = df['category_4'].fillna(2)
    df['category_4'] = df['category_4'].map({'Y': 0, 'N': 1, 2 : 2}).astype(int)

    df['category_2'] = df['category_2'].fillna(6)
    df['category_2'] = df['category_2'].map({1.0 : 1, 2.0 : 2, 3.0 : 3,4.0 : 4, 5.0 : 5, 6 : 6}).astype(int)

    #for missing id we will use -1111 as filling value 
    #find the number of missing values 
    for col in df[['merchant_group_id','state_id',
                            'merchant_category_id','subsector_id','city_id']]:
                            df[col] = df[col].fillna(-1111)

    #imputing the inf values with max 
    features_inf = ["avg_purchases_lag3","avg_purchases_lag6","avg_purchases_lag12"]
    for col in features_inf:
        df.loc[df[col]==np.inf,col] = max(df.loc[df[col]!=np.inf,col])

    # reduce memory usage
    #df = reduce_mem_usage(df)

    return df

## FE and Grouping on card id 

In [51]:
def group_on_card_id(df):
    """
    FUNCTION:
    To group the data on card id  

    ARGS:
    df is the data frame on which grouping needs to be performed 

    RETURNS:
    returns the data frame after grouping 
    
    """

    #Some feature engineering on date and time  
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    df['hour'] = df['purchase_date'].dt.hour
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['weekday'] = df['purchase_date'].dt.weekday
    df['weekend'] = (df['purchase_date'].dt.weekday >= 5).astype(int)
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']


    mode = lambda x: x.value_counts().index[0]

    agg_fun = {
        
    "authorized_flag": ['sum', 'mean',mode],
    'category_1' : ['sum', 'mean',mode],
    'category_2' :['sum', 'mean',mode],
    'category_3' :['sum', 'mean',mode],
    'category_1_merchants_t' :['sum', 'mean',mode],
    'most_recent_sales_range_merchants_t' :['sum', 'mean',mode],
    'category_4_merchants_t' :['sum', 'mean',mode],
    'most_recent_purchases_range_merchants_t' :['sum', 'mean',mode],
    'category_2_merchants_t' :['sum', 'mean',mode],


    'city_id' : ['nunique',mode],
    'state_id' : ['nunique',mode],
    'subsector_id' : ['nunique',mode],
    'merchant_category_id' : ['nunique',mode],
    'merchant_id': ['nunique',mode],
    'merchant_group_id_merchants_t':['nunique',mode],
    'state_id_merchants_t':['nunique',mode],
    'merchant_category_id_merchants_t':['nunique',mode],
    'subsector_id_merchants_t':['nunique',mode],
    'city_id_merchants_t':['nunique',mode],


    'month_lag' : ['sum', 'mean', 'min', 'max', 'std','var'],
    'installments' : ['sum', 'mean', 'min', 'max', 'std','var'],
    'purchase_amount' : ['sum', 'mean', 'min', 'max', 'std','var'],
    'numerical_2_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_sales_lag3_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_purchases_lag3_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'active_months_lag3_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_sales_lag6_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_purchases_lag6_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_sales_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_purchases_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'numerical_1_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],

    'active_months_lag6_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'active_months_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'active_months_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],


    'weekend': ['nunique', 'mean', 'min', 'max'],
    'weekday' : ['nunique', 'mean', 'min', 'max'],
    'hour': ['nunique', 'mean', 'min', 'max'],
    'weekofyear': ['nunique', 'mean', 'min', 'max'],
    'day': ['nunique', 'mean', 'min', 'max'],
    #Refer:-https://numpy.org/doc/stable/reference/generated/numpy.ptp.html
    'purchase_date' : [np.ptp, 'min', 'max'],
    'month' : ['sum', 'mean', 'nunique']
    }

    df = df.groupby("card_id",as_index=False).agg(agg_fun)
    print(df.card_id)

    # reduce memory usage
    df = reduce_mem_usage(df)

    return df

## Group on merchant id 

In [52]:
def group_on_merchant_id(df):
    """
    FUNCTION:
    To group the data on merchant id 

    ARGS:
    df is the data frame on which grouping needs to be performed 

    RETURNS:
    returns the data frame after grouping 
    
    """


    mode = lambda x:x.value_counts().index[0]
    
    agg_fun = {
    "merchant_group_id": mode,
    "merchant_category_id": mode,
    "subsector_id": mode,
    "numerical_1": "mean",
    "numerical_2": "mean",
    "category_1": mode,
    "most_recent_sales_range": mode,
    "most_recent_purchases_range": mode,
    "avg_sales_lag3": "mean",
    "avg_purchases_lag3": "mean",
    "active_months_lag3": mode,
    "avg_sales_lag6": "mean",
    "avg_purchases_lag6": "mean",
    "active_months_lag6": mode,
    "avg_sales_lag12": "mean",
    "avg_purchases_lag12": "mean",
    "active_months_lag12": mode,
    "category_4": mode,
    "city_id": mode,
    "state_id": mode,
    "category_2": mode
    }

    df = df.groupby("merchant_id",as_index=False).agg(agg_fun)
    print(df.merchant_id)

    df.columns = [col+"_merchants_t" if col!="merchant_id" else col for col in df.columns]


    # reduce memory usage
    df = reduce_mem_usage(df)

    return df

## aug 

In [53]:
def generate_augmented_train_test_1(debug=False):
    """ Generate train and test augmented datasets. """
    num_rows = 100 if debug else None
    with timer("train & test"):
        df = train_test(num_rows).reset_index()
    with timer("transactions"):
        # load csv
        new_merchant_df = pd.read_csv('/content/drive/MyDrive/data/new_merchant_transactions.csv', nrows=num_rows)
        historical_transactions_df = pd.read_csv('/content/drive/MyDrive/data/historical_transactions.csv', nrows=num_rows)
        df_1 = pd.concat([historical_transactions_df, new_merchant_df], ignore_index=True)
        df_1 = transactions_imputations(df_1)
        df = pd.merge(df, df_1, right_on='card_id',left_on='df_card_id_', how='left')
        del new_merchant_df
        del historical_transactions_df
        gc.collect()
    with timer("split train & test"):
        train_df = df[df['target'].notnull()]
        test_df = df[df['target'].isnull()]
        del test_df['target']
        del df
        gc.collect()
    with timer("Save train and test files"):

        train_df.to_csv('/content/drive/MyDrive/data/augmented_train.csv', index=False)
        test_df.to_csv('/content/drive/MyDrive/data/augmented_test.csv', index=False)

## Generating the aurgumentated dataset 

In [54]:
def generate_augmented_train_test(debug=False):
    """ 
    Generate train and test augmented datasets. 
    STEPS:-
    1. FIRST CONCATINATE 
    2. DO THE IMPUTATIONS OF DATA 
    3. LOAD MERCHANTS 
    4. GROUP ON MERCHANT ID 
    5. MERGE THE TRANSACTIONS WITH MERCHANTS 
    6. GROUP ON CARD ID 
    7. PREPARE THE MAIN DATA SET
    8. SPLIT TRAIN AND TEST 
    9. SAVE THE ARGUMENTED TRAIN AND TEST IN CSV 
    returns train and test in csv format 
    """
    num_rows = 100 if debug else None
    with timer("transactions"):
        # load csv
        print("[INFO] LOADING NEWMERCHANT TRANSACTIONS AND HISTORICAL TRANSACTIONS ....")
        new_merchant_df = pd.read_csv('/content/drive/MyDrive/data/new_merchant_transactions.csv', nrows=num_rows)
        historical_transactions_df = pd.read_csv('/content/drive/MyDrive/data/historical_transactions.csv', nrows=num_rows)
        print("[INFO] CONCATINATING THE TRANSACTIONS DATA ....")
        df = pd.concat([historical_transactions_df, new_merchant_df], ignore_index=True)
        print("[INFO] DELETING THE DATA FRAMES ....")
        del new_merchant_df
        del historical_transactions_df
        print("The concatinated data frame is ",df.card_id)
        #Do the imputation 
        print("[INFO] DOING THE TRANSACTIONS IMPUTATIONS ...")
        df = transactions_imputations(df)
        print("[INFO] THE DATA SET IS ... \n",df.card_id)
        gc.collect()
    with timer("merchants"):
        #load the data and do the imputations 
        print("[INFO] LOADING MERCHANT DATA AND DOING IMPUTATIONS OF THAT ")
        merchants_df = merchant_imputations(num_rows).reset_index()
        print("[INFO] PRINTING MERCHANTS COLUMNS ...")
        print(merchants_df.columns)
        print("[INFO] PRINTING MAIN DATA FRAME COLUMNS ...")
        print(df.columns)
        #group by merhcant data
        print("[INFO] GROUPING ON MERHCHANT ID ...")
        merchants_df = group_on_merchant_id(merchants_df)
        #group by merhcant data
        print("[INFO] PRINTING MERHCHANT DATA  COLUMNS...",merchants_df.columns)
        print("[INFO] MERGING MERCHANTS WITH TRANSACRIONS  ...")
        df = pd.merge(df, merchants_df, on='merchant_id', how='left')
        print(df.columns)
        print(df.isnull().sum(axis = 0))
    with timer("group_on_card_id"):
        print("[INFO] GROUPING ON CARD ID ...")
        df = group_on_card_id(df)
    with timer("train & test"):
        print("[INFO] LOADING THE TRAIN AND TEST ...")
        tt = train_test(num_rows).reset_index()
        print(tt.columns)
        print(df.columns)
        print(df.df_card_id_)
        print(tt.card_id)
        print("[INFO] MERGING THE DTAAFRAMES ....")
        df = pd.merge(df, tt,left_on ='df_card_id_',  right_on='card_id', how='right')
    with timer("split train & test"):
        print("[INFO] SPLITING THE DATASET ....")
        train_df = df[df['target'].notnull()]
        test_df = df[df['target'].isnull()]
        del test_df['target']
        del df
        gc.collect()
    with timer("Save train and test files"):
        train_df.to_csv('/content/drive/MyDrive/data/augmented_train.csv', index=False)
        test_df.to_csv('/content/drive/MyDrive/data/augmented_test.csv', index=False)


## Main Method

In [55]:
if __name__ == "__main__":
  generate_augmented_train_test(True)

[INFO] LOADING NEWMERCHANT TRANSACTIONS AND HISTORICAL TRANSACTIONS ....
[INFO] CONCATINATING THE TRANSACTIONS DATA ....
[INFO] DELETING THE DATA FRAMES ....
The concatinated data frame is  0      C_ID_4e6213e9bc
1      C_ID_4e6213e9bc
2      C_ID_4e6213e9bc
3      C_ID_4e6213e9bc
4      C_ID_4e6213e9bc
            ...       
195    C_ID_0a6ceeffcc
196    C_ID_0a6ceeffcc
197    C_ID_0a6ceeffcc
198    C_ID_0a6ceeffcc
199    C_ID_0a6ceeffcc
Name: card_id, Length: 200, dtype: object
[INFO] DOING THE TRANSACTIONS IMPUTATIONS ...
[INFO] THE DATA SET IS ... 
 0      C_ID_4e6213e9bc
1      C_ID_4e6213e9bc
2      C_ID_4e6213e9bc
3      C_ID_4e6213e9bc
4      C_ID_4e6213e9bc
            ...       
195    C_ID_0a6ceeffcc
196    C_ID_0a6ceeffcc
197    C_ID_0a6ceeffcc
198    C_ID_0a6ceeffcc
199    C_ID_0a6ceeffcc
Name: card_id, Length: 200, dtype: object
transactions - done in 0s
[INFO] LOADING MERCHANT DATA AND DOING IMPUTATIONS OF THAT 
[INFO] PRINTING MERCHANTS COLUMNS ...
Index(['index', 'merc

IndexError: ignored

## Load the dataset 

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/data/augmented_train.csv')

## Start Exploring 

In [ ]:
train_df.isnull().sum(axis = 0)

df_card_id_                      0
df_authorized_flag_sum           0
df_authorized_flag_mean          0
df_authorized_flag_<lambda_0>    0
df_category_1_sum                0
                                ..
feature_sum                      0
feature_mean                     0
feature_max                      0
feature_min                      0
feature_std                      0
Length: 89, dtype: int64

In [ ]:
train_df

,df_card_id_,df_authorized_flag_sum,df_authorized_flag_mean,df_authorized_flag_<lambda_0>,df_category_1_sum,df_category_1_mean,df_category_1_<lambda_0>,df_category_2_sum,df_category_2_mean,df_category_2_<lambda_0>,df_category_3_sum,df_category_3_mean,df_category_3_<lambda_0>,df_city_id_nunique,df_city_id_<lambda_0>,df_state_id_nunique,df_state_id_<lambda_0>,df_subsector_id_nunique,df_subsector_id_<lambda_0>,df_merchant_category_id_nunique,df_merchant_category_id_<lambda_0>,df_merchant_id_nunique,df_merchant_id_<lambda_0>,df_month_lag_sum,df_month_lag_mean,df_month_lag_min,df_month_lag_max,df_month_lag_std,df_month_lag_var,df_installments_sum,df_installments_mean,df_installments_min,df_installments_max,df_installments_std,df_installments_var,df_purchase_amount_sum,df_purchase_amount_mean,df_purchase_amount_min,df_purchase_amount_max,df_purchase_amount_std,...,df_weekend_min,df_weekend_max,df_weekday_nunique,df_weekday_mean,df_weekday_min,df_weekday_max,df_hour_nunique,df_hour_mean,df_hour_min,df_hour_max,df_weekofyear_nunique,df_weekofyear_mean,df_weekofyear_min,df_weekofyear_max,df_day_nunique,df_day_mean,df_day_min,df_day_max,df_purchase_date_ptp,df_purchase_date_min,df_purchase_date_max,df_month_sum,df_month_mean,df_month_nunique,card_id,first_active_month,feature_1,feature_2,feature_3,target,outliers,quarter,elapsed_time,quarter_first_active_month,first_active_month_diff_from_today,feature_sum,feature_mean,feature_max,feature_min,feature_std
0,C_ID_92a2005557,270,0.954064,1,0,0.000000,0,295,1.042403,1,4,0.014134,0,9,69,3,9,21,34,46,560,117,M_ID_1a81c358a3,-983,-3.473498,-8,2,2.734435,7.477132,4.0,0.014134,0.0,1.0,0.118254,0.013984,-180.671336,-0.638415,-0.739395,0.800000,0.143774,...,0,1,7,3.204947,0,6,23,13.279152,0,23,42,31.466431,1,52,31,15.586572,1,31,305 days 21:04:57,2017-06-27 14:18:08,2018-04-29 11:23:05,2175,7.685512,11,C_ID_92a2005557,2017-06-01 00:00:00,1.0,1.0,1.0,-0.820283,1.0,2.0,1292.0,2.0,1292.0,3.0,1.0,1.0,1.0,0.0
1,C_ID_3d0044924f,345,0.969101,1,31,0.087079,0,511,1.435393,1,432,1.213483,1,9,69,3,9,24,34,58,307,148,M_ID_940fb4498f,-1752,-4.921348,-12,2,3.865994,14.945909,551.0,1.547753,0.0,10.0,1.491939,2.225882,-219.750730,-0.617277,-0.742400,0.800000,0.240444,...,0,1,7,3.331461,0,6,24,14.657303,0,23,50,24.946629,1,52,31,16.612360,1,31,447 days 14:18:44,2017-01-06 16:29:42,2018-03-30 06:48:26,2192,6.157303,12,C_ID_3d0044924f,2017-01-01 00:00:00,1.0,1.0,1.0,0.392913,1.0,1.0,1443.0,1.0,1443.0,3.0,1.0,1.0,1.0,0.0
2,C_ID_d639edf6cd,42,0.954545,1,0,0.000000,0,204,4.636364,5,0,0.000000,0,5,143,2,5,8,33,9,705,14,M_ID_5634fd83e0,-368,-8.363636,-13,2,4.120798,16.980973,0.0,0.000000,0.0,0.0,0.000000,0.000000,-29.867717,-0.678812,-0.730138,-0.145847,0.086422,...,0,1,7,3.340909,0,6,14,17.886364,8,23,22,18.340909,2,49,19,19.522727,2,30,472 days 09:21:49,2017-01-11 08:21:22,2018-04-28 17:43:11,200,4.545455,10,C_ID_d639edf6cd,2016-08-01 00:00:00,1.0,1.0,1.0,0.688056,1.0,3.0,1596.0,3.0,1596.0,3.0,1.0,1.0,1.0,0.0
3,C_ID_186d6a6901,84,1.000000,1,13,0.154762,0,326,3.880952,4,97,1.154762,1,7,17,5,22,15,37,28,278,57,M_ID_00a6ca8a8a,-206,-2.452381,-5,2,2.141977,4.588067,92.0,1.095238,0.0,3.0,0.505846,0.255881,-54.791332,-0.652278,-0.740897,0.800000,0.188341,...,0,1,7,2.833333,0,6,17,14.321429,0,23,25,30.500000,1,52,26,16.559524,2,31,203 days 18:37:50,2017-09-26 16:22:21,2018-04-18 11:00:11,622,7.404762,8,C_ID_186d6a6901,2017-09-01 00:00:00,1.0,1.0,1.0,0.142495,1.0,3.0,1200.0,3.0,1200.0,3.0,1.0,1.0,1.0,0.0
4,C_ID_cdbd2c0db2,164,0.970414,1,17,0.100592,0,668,3.952663,4,179,1.059172,1,7,17,7,22,19,37,37,278,102,M_ID_cecefd9589,-115,-0.680473,-3,2,1.497509,2.242533,218.0,1.289941,0.0,12.0,1.691704,2.861862,-95.172168,-0.563149,-0.746156,0.800000,0.303462,...,0,1,7,3.248521,0,6,22,13.402367,0,23,24,19.988166,1,52,31,15.775148,1,31,167 days 18:50:25,2017-11-12 00:00:00,2018-04-28 18:50:25,847,5.011834,6,C_ID_cdbd2c0db2,2017-11-01 00:00:00,1.0,1.0,1.0,-0.159749,1.0,4.0,1139.0,4.0,1139.0,3.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,.

In [ ]:
train_df.card_id

0         C_ID_92a2005557
1         C_ID_3d0044924f
2         C_ID_d639edf6cd
3         C_ID_186d6a6901
4         C_ID_cdbd2c0db2
               ...       
201912    C_ID_963962de2c
201913    C_ID_1314773c0b
201914    C_ID_7666735b3d
201915    C_ID_73f5a0efd0
201916    C_ID_92c9984c58
Name: card_id, Length: 201917, dtype: object

In [ ]:
train_df.columns

Index(['df_card_id_', 'df_authorized_flag_sum', 'df_authorized_flag_mean',
       'df_authorized_flag_<lambda_0>', 'df_category_1_sum',
       'df_category_1_mean', 'df_category_1_<lambda_0>', 'df_category_2_sum',
       'df_category_2_mean', 'df_category_2_<lambda_0>', 'df_category_3_sum',
       'df_category_3_mean', 'df_category_3_<lambda_0>', 'df_city_id_nunique',
       'df_city_id_<lambda_0>', 'df_state_id_nunique',
       'df_state_id_<lambda_0>', 'df_subsector_id_nunique',
       'df_subsector_id_<lambda_0>', 'df_merchant_category_id_nunique',
       'df_merchant_category_id_<lambda_0>', 'df_merchant_id_nunique',
       'df_merchant_id_<lambda_0>', 'df_month_lag_sum', 'df_month_lag_mean',
       'df_month_lag_min', 'df_month_lag_max', 'df_month_lag_std',
       'df_month_lag_var', 'df_installments_sum', 'df_installments_mean',
       'df_installments_min', 'df_installments_max', 'df_installments_std',
       'df_installments_var', 'df_purchase_amount_sum',
       'df_purchase_a

In [ ]:
train_df['df_category_3_<lambda_0>']

0         0
1         1
2         0
3         1
4         1
         ..
201912    0
201913    0
201914    1
201915    0
201916    1
Name: df_category_3_<lambda_0>, Length: 201917, dtype: int64

### Checking for percentage of null values 

In [ ]:
#Gives the name of all the variables with missing data 

columns_with_na = [var for var in train_df.columns if train_df[var].isnull().mean()  > 0]

In [ ]:
len(columns_with_na)

1

In [ ]:
#lets find out the percentage of observations missing per variable

#calculate the percentage of missing 
data_na = train_df[columns_with_na].isnull().mean()

#transform the array to dataframe 
data_na = pd.DataFrame(data_na.reset_index())

#add names to the dataframe 
data_na.columns = ['col','percentage_na']

#oreder the dataframe acc to percentage 
data_na.sort_values(by = 'percentage_na',ascending = False, inplace = True)

#show
data_na

,col,percentage_na
0,df_price_skew,0.000045


In [ ]:
!ls

 augmented_test.csv			  new_merchant_transactions.csv
 augmented_train.csv			  sample_submission.csv
'Data Dictionary.xlsx'			  test.csv
 Data_Dictionary.xlsx			  train.csv
 historical_transactions.csv		  transactions_raw_merged.csv
 merchants.csv				  transactions_refined_1_merged_WON.csv
 merged_transactions_with_merchants.csv


In [ ]:
def lgb_train_fn(train_df, target, trn_cols,  n_fold):
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=4590)
    # initialise out of fold preds to 0s.
    oof = np.zeros(len(train_df))

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df,train_df['outliers'].values)):
        trn_data = lgb.Dataset(train_df.iloc[trn_idx][trn_cols], label=target.iloc[trn_idx])
        val_data = lgb.Dataset(train_df.iloc[val_idx][trn_cols], label=target.iloc[val_idx])

        num_round = 10000
        clf = lgb.train(lgb_param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=0, early_stopping_rounds = 200)
        oof[val_idx] = clf.predict(train_df.iloc[val_idx][trn_cols], num_iteration=clf.best_iteration)

    print(np.sqrt(mean_squared_error(oof, target)), 'CV score')
    return np.sqrt(mean_squared_error(oof, target))

In [ ]:
for c in cols_to_add:
    lgb_cols = final_cols + [c]
    print(len(lgb_cols), 'lg_cols', c)
    score = lgb_train_fn(x, y, lgb_cols, 5)
    delta = base_score - score
    fe_d[c] = delta
    if delta > 0:
        base_score = score
        selected_cols.append(c)
        print('Selected cols', c)
        print('Selected col delta', delta)
        print(' score with col', score)
        np.save('selecte_cols_extra', selected_cols)
        final_cols = final_cols + [c]